<a href="https://colab.research.google.com/github/praths007/tensorflow_certification/blob/master/coursera_practise/tensorflow_NLP_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf

In [4]:
tf.__version__

'2.3.0'

In [5]:
import tensorflow

In [32]:
import numpy as np
import csv
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, GlobalAvgPool1D, Dense

In [8]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/bbc-text.csv \
    -O /tmp/bbc-text.csv

--2020-08-17 10:33:29--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/bbc-text.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.204.128, 172.217.203.128, 74.125.141.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.204.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5057493 (4.8M) [application/octet-stream]
Saving to: ‘/tmp/bbc-text.csv’

/tmp/bbc-text.csv   100%[===================>]   4.82M  --.-KB/s    in 0.05s   

2020-08-17 10:33:29 (106 MB/s) - ‘/tmp/bbc-text.csv’ saved [5057493/5057493]



In [9]:
# pd.read_csv("/tmp/bbc-text.csv")

In [101]:
label = []
text = []
with open("/tmp/bbc-text.csv", "r") as csv_file:
  file_csv = csv.reader(csv_file, delimiter=',')
  # print(file_csv)
  header = 1
  for row in file_csv:
    if(header != 1):
      label.append(row[0])
      text.append(row[1])
    else:
      header+=1
      continue

In [139]:
train_size = int(len(label) * 0.8)
train_x = text[:train_size]
train_y = label[:train_size]

test_x = text[train_size:]
test_y = label[train_size:]

In [140]:
vocab_size = 500
embedding_dims = 10000
output_dims = 64
max_len = 100

In [141]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(train_x)

label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(train_y)
train_y = label_tokenizer.texts_to_sequences(train_y)
test_y = label_tokenizer.texts_to_sequences(test_y)

train_x_seq = tokenizer.texts_to_sequences(train_x)
train_x_padded = pad_sequences(train_x_seq, maxlen=max_len)

test_x_seq = tokenizer.texts_to_sequences(test_x)
test_x_padded = pad_sequences(test_x_seq, maxlen=max_len)

train_x_padded = np.array(train_x_padded)
test_x_padded = np.array(test_x_padded)
train_y = np.array(train_y)
test_y = np.array(test_y)

In [142]:
len(train_x_padded)

1780

In [143]:
train_x_padded.shape

(1780, 100)

In [144]:
model = Sequential([
                    Embedding(input_dim=embedding_dims, output_dim=output_dims, input_length=train_x_padded.shape[1]),
                    GlobalAvgPool1D(),
                    Dense(64, activation='relu'),
                    Dense(6, activation='softmax')
])



In [145]:
model.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (None, 100, 64)           640000    
_________________________________________________________________
global_average_pooling1d_20  (None, 64)                0         
_________________________________________________________________
dense_40 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_41 (Dense)             (None, 6)                 390       
Total params: 644,550
Trainable params: 644,550
Non-trainable params: 0
_________________________________________________________________


In [146]:
np.unique(train_y), np.unique(test_y)

(array([1, 2, 3, 4, 5]), array([1, 2, 3, 4, 5]))

In [149]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

model.fit(train_x_padded, train_y, validation_data=(test_x_padded, test_y), epochs=30)

Epoch 1/30
56/56 [==============================] - 1s 10ms/step - loss: 0.2732 - acc: 0.9281 - val_loss: 0.4729 - val_acc: 0.8427
Epoch 2/30
56/56 [==============================] - 0s 8ms/step - loss: 0.2507 - acc: 0.9360 - val_loss: 0.4634 - val_acc: 0.8360
Epoch 3/30
56/56 [==============================] - 0s 8ms/step - loss: 0.2271 - acc: 0.9399 - val_loss: 0.4469 - val_acc: 0.8427
Epoch 4/30
56/56 [==============================] - 0s 8ms/step - loss: 0.2087 - acc: 0.9449 - val_loss: 0.4361 - val_acc: 0.8517
Epoch 5/30
56/56 [==============================] - 0s 8ms/step - loss: 0.1931 - acc: 0.9489 - val_loss: 0.4310 - val_acc: 0.8539
Epoch 6/30
56/56 [==============================] - 0s 8ms/step - loss: 0.1773 - acc: 0.9478 - val_loss: 0.4300 - val_acc: 0.8382
Epoch 7/30
56/56 [==============================] - 0s 8ms/step - loss: 0.1642 - acc: 0.9528 - val_loss: 0.4300 - val_acc: 0.8404
Epoch 8/30
56/56 [==============================] - 0s 8ms/step - loss: 0.1526 - acc: 0.9